# Object Detection and Segmentation

## Image Segmentation


Object classification focused on making categories and detection focused on looking for an object in a given scene. Separate to the above requirements there are use cases where we need to separate the items/objects of a scene to various classes rather than just recognizing and finding the bounding box. The name of that process is called segmentation. What basically happen here is all the pixels of the scene will get assigned a unique identifier to separate out same object and background.

---

### **Masked R-CNN and Instance Segmentation**

Earlier we discussed about the YOLO and RetinaNet models. These models are called single shot detectors as they only need to traverse the image once to detect the required object. Other than this approach there is another technique which use 2 neural networks. It uses the first neural network to suggest a potential location for the object to be detected, then use the second network to clssify and fine tune the boundary boxes. This type of architecture is called `Region Proposal Networks (RPNs)`.

But as we can imagine RPNs tend to be more complex and slower compared to the single shot detectors. On the other hand they are more accurate as well. Because of that reason there are many variant of RPNs like R-CNN, fast R-CNN and faster R-CNN. One of the interesting such RPN type model is masked R-CNN. 

One main reason for considering this model is not because it perform slightly better compared to previously mentioned RPNs, but its ability to perform instance segmentation tasks with slight extensions. In fact it can be extended to find every pixel belonging to the detected object. 

### Region Proposal Networks (RPNs)

RPN is a type of simplified single shot detection network that only cares about 2 classes the object (foreground or anything that is labeled in the dataset) and background. It can use an architecture similar to the RetinaNet and will predict the bounding boxes and their classification (object or background) using the 2 prediction heads.

It will have a slightly modified loss calculation function since we only care about 2 main classes object or background. Also just as in the RetinaNet predicted boxes will go through non max suppression process. And the left boxes are considered as the box proposal or Region Of Interests (ROIs) for the next stage which is the actual segmentation. 

It should also be noted that selected ROIs from this stage can still be rejected by the next stage as background, so having perfect accuracy is not essential. Also we can use any network that can predict classification and rough ROI for the object of interest which will be refined in the next step.


### R-CNN

The basic idea of the R-CNN is to get the ROIs cropped from original input and then rerun it through the backbone model to get classifications to the selected objects. But in practice this is slow/redundand process and therefore instead we directly use the related feature maps of the ROIs. This way we can avoid the rerunning the cropped image parts separately through the backbone network. This process get bit more complicated if we are using FPNs. Since ROIs can come from different levels we need to choose the proper FPN level. Also the authors of Mask R-CNN paper noted that using rounded values for the feature map widths and heights of FPN may degrade the performance of the model.

Once we figure out the feature maps for the ROIs rest is straight forward. We just need to send the extracted feature through the predictions heads to get the classifications and precise box refinements. Loss calculation and other related parts are almost similar to the RetinaNet.

The difference of Masked R-CNN is it has a third classification head to classify the individual pixels of objects. The result is a pixel mask that outline the object. But to train this part we need training data with objects masks as well. Before going in to more details of the masked head, we need to look at special operation used in Masked R-CNN for pixel generation called `Transposed Convolutions`.

#### Transposed Convolutions

In simple terms transposed convolutions is an upsampling operation. Difference between this technique and regular upsampling techniques is that former having learnable weights. 

<center><image src="./imgs/17.png" width="500px"/></center>

As we can see in the above image, for every input value (in the top) will get multiplied by a filter and applied to the output. The stride defined in the filter movement makes the upsampling happen. Also if input have multiple channels filter would also have that number of channels and do the operation. Then will take the element wise summation before adding to the final output. If we need to have multiple output channels then we need to have multiple filters.

> It should be noted that this technique is known to cause checker board like artifacts in the output. Therefore later researchers introduced `up-convolutions` technique which have a basic neighbour sampling followed by a typical convolution layer that can reduce the mentioned issue.